# Air Quality and Social Justice

In the Salt Lake area, where smog will periodically hide the mountains that surround the city, [air pollution](http://slcair.communication.utah.edu/) is a consistent concern. The smog is a result of the pollution trapped by the unique topographic features of the Wasatch area. In this notebook, we will explore the relationship between air quality (pm25 and ozone) and income and elevation in the Salt Lake valley. This will allow us to explore questions related to social justice such as, "Is air pollution exposure equally distributed across different socio-economic groups?"

### Import packages that we are going to use
#### These include

* [Pandas](http://pandas.pydata.org/index.html): A package for reading and manipulating tabular data.
* [Bokeh](https://bokeh.pydata.org/en/latest/): Bokeh is a visualization library that provides versatile graphics and interactivity.
* [Folium](http://python-visualization.github.io/folium/docs-v0.5.0/): Folium is a good package for visualizing geographic data.
* [statsmodels](http://www.statsmodels.org/stable/): This package provides a number of common statistical functions for analysing data.


In [15]:
import pandas as pd
import statsmodels
import statsmodels.formula.api as smf
import patsy
import numpy as np
#import os
#import folium

In [16]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import Viridis256
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

### What are these data

In our data directory we have two **csv** files (csv stands for **C**omma **S**eparated **V**alues). For different regions in the Salt Lake valley (zip code), we have computed the average

1. Income
1. Elevation
1. pm25 levels for March 8, 2016
1. ozone levels at 10:00 am and 3:00 PM from August 2016.

These files have been created by Dr. Daniel L. Mendoza at the University of Utah.

## Use Pandas to read in the data

Pandas reads the data into a Pandas **dataframe** which we assigning to the variable ``pm25``. Dataframes have two methods for looking at the data, ``head()`` and ``tail()``, to look at the first and last rows, respectively.

In [17]:
pm25 = pd.read_csv("Class_PM25_Data.csv")
pm25.tail()

,Block_Group_ID,Income,Elevation,PM25_MAR_8
60,4.903510e+11,69188.0,1321.630859,4.409883
61,4.903510e+11,69493.0,1312.114014,3.989309
62,4.903510e+11,92551.0,1557.853394,1.537066
63,4.903510e+11,60817.0,1552.666016,2.252555
64,4.903600e+11,NaN,1275.227905,3.717521


#### Notice the ``NaN`` value for Income in our last data row

Pandas uses ``NaN`` (Not a number) to represent missing values. That is our data file did not have an income value for the last row. There are a variety of approaches for dealing with missing data, but we are just going to drop that value using the Pandas dataframe ``dropna`` method.



In [18]:
pm25 = pm25.dropna()
pm25.tail()

,Block_Group_ID,Income,Elevation,PM25_MAR_8
59,4.903510e+11,35375.0,1305.029541,3.595146
60,4.903510e+11,69188.0,1321.630859,4.409883
61,4.903510e+11,69493.0,1312.114014,3.989309
62,4.903510e+11,92551.0,1557.853394,1.537066
63,4.903510e+11,60817.0,1552.666016,2.252555


### Now let's repeat this with the ozone data

In [19]:
ozone = pd.read_csv("Class_Ozone_Data.csv")
ozone = ozone.dropna()
ozone.tail()

,Block_Group_ID,Income,Elevation,Ozone_AUG_10,Ozone_AUG_15
59,490351140001,35375.0,1305.029541,22.561118,48.027862
60,490351142001,69188.0,1321.630859,30.320732,57.617152
61,490351142002,69493.0,1312.114014,31.675820,59.208955
62,490351152091,92551.0,1557.853394,36.326890,58.199097
63,490351152092,60817.0,1552.666016,36.182543,57.752619


## Exploratory Plots

In statistics, it is always a good idea to start by plotting your data to get an idea of what it looks like. This can help you determine which analyses would be useful to perform. Here we are going to be using Bokeh to do some exploratory visualization.

Our first plot is going to be a scatterplot to visualize the relationships between elevation, income, and PM2.5 exposure. We've used a colorbar to show the differences in elevation. Hover over the points to reveal the exact elevation.

In [46]:
bpm25 = ColumnDataSource(pm25)

color_mapper = LinearColorMapper(palette=Viridis256, low=pm25.Elevation.min(), high=pm25.Elevation.max())
hover = HoverTool(tooltips=[('elevation', '@Elevation')])

p1 = figure(tools=[hover,'pan','wheel_zoom'],
           plot_width=900, plot_height=400, x_axis_label='Income',
           y_axis_label='PM2.5 Level')

p1.circle(x='Income', y='PM25_MAR_8', color={'field': 'Elevation', 'transform': color_mapper}, 
         size=20, alpha=0.6, source=bpm25)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Elevation')
p1.add_layout(color_bar, 'right')
show(p1)

It looks like PM2.5 levels are higher for low incomes and the majority of low elevations. The left side of the chart shows lower incomes and elevations with higher PM2.5 exposure, while the right side has higher incomes and elevations with lower PM2.5 levels. Let's do some regressions to quantify the strength of these relationships.

We will use ``statsmodels`` to do [ordinary least squares regression](https://en.wikipedia.org/wiki/Ordinary_least_squares).

```Python
mod = smf.ols(formula='PM25_MAR_8 ~ Income', data=pm25)
```

* This formula is stating a linear relationship between our dependent variable, ``PM25_MAR_8``, and our independent variable, ``Income``.

```Python
'PM25_MAR_8 ~ Income'
```
```Python
data=pm25
```

* Use the ``pm25`` dataset
* ``mod.fit()`` fits the model to the data
* ```res.summary()``` provides a detailed report on how well the model fit the data.

In [21]:
mod = smf.ols(formula='PM25_MAR_8 ~ Income', data=pm25)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             PM25_MAR_8   R-squared:                       0.458
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     52.32
Date:                Fri, 08 Jun 2018   Prob (F-statistic):           8.48e-10
Time:                        15:51:12   Log-Likelihood:                -77.678
No. Observations:                  64   AIC:                             159.4
Df Residuals:                      62   BIC:                             163.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.8552      0.256     18.981      0.000       4.344       5.367
Income      -3.74e-05   5.17e-06     -7.233      0.000   -4.77e-05   -2.71e-05
==============================================================================
Omnibus:                        6.338   Durbin-Watson:                   1.194
Prob(Omnibus):                  0.042   Jarque-Bera (JB):                5.446
Skew:                           0.624   Prob(JB):                       0.0657
Kurtosis:                       3.697   Cond. No.                     1.22e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### This is a lot of information. What are the key points?

#### Let's start by looking at our overall model.

* **Prob (F-statistic):**           8.48e-10 ($8.48e^{-10}$). 
    * This is the probability that the linear relationship between our variables is purely due to chance. 
* **R-squared:**                       0.458
    * This is the proportion of the variability in our data that is explained by our model, so about half of the variability in the data is explained by income.
* **Cond. No.**                     1.22e+05 ($1.22e^{05}$)
    * A large condition number indicates numeric problems with our model/data and mean the results are less reliable.

#### Now let's look at our ``Income`` variable

* **coef**=-3.74e-05 ($-3.74e^{-05}$). This is the slope of the line. The slope is negative, meaning as income **increases**, air pollution **decreases.**
* **P**=0.000. This is the "p-value" and describes the probability that the linear relationship is just random chance. Since our p-value is less than 0.001, we can accept our hypothesis that income and pollution are related.

## Let's repeat this for the relationship between ``Income`` and ``Elevation``

In [22]:
mod = smf.ols(formula='Income ~ Elevation', data=pm25)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Income   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     35.58
Date:                Fri, 08 Jun 2018   Prob (F-statistic):           1.28e-07
Time:                        15:51:16   Log-Likelihood:                -710.14
No. Observations:                  64   AIC:                             1424.
Df Residuals:                      62   BIC:                             1429.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.152e+05   4.37e+04     -4.923      0.000   -3.03e+05   -1.28e+05
Elevation    194.0279     32.529      5.965      0.000     129.003     259.053
==============================================================================
Omnibus:                        1.675   Durbin-Watson:                   0.978
Prob(Omnibus):                  0.433   Jarque-Bera (JB):                1.047
Skew:                          -0.285   Prob(JB):                        0.592
Kurtosis:                       3.262   Cond. No.                     2.90e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.9e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [39]:
#add regression line
p = figure(plot_width=400, plot_height=400, x_axis_label='Income', y_axis_label='Elevation')
p.circle(x='Income', y='Elevation', 
         size=20, alpha=0.6, color='orange', source=bpm25)

show(p)

### The relationship between ``Income`` and ``Elevation`` is strong
### Let's use ``Elevation`` to predict ``PM25_MAR_8``

In [23]:
mod = smf.ols(formula='PM25_MAR_8 ~ Elevation', data=pm25)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             PM25_MAR_8   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     50.84
Date:                Fri, 08 Jun 2018   Prob (F-statistic):           1.28e-09
Time:                        15:51:22   Log-Likelihood:                -78.095
No. Observations:                  64   AIC:                             160.2
Df Residuals:                      62   BIC:                             164.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.1668      2.247      8.531      0.000      14.675      23.658
Elevation     -0.0119      0.002     -7.130      0.000      -0.015      -0.009
==============================================================================
Omnibus:                        1.175   Durbin-Watson:                   0.842
Prob(Omnibus):                  0.556   Jarque-Bera (JB):                0.988
Skew:                           0.013   Prob(JB):                        0.610
Kurtosis:                       2.392   Cond. No.                     2.90e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.9e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
#add regression line
p2 = figure(plot_width=400, plot_height=400, x_axis_label='Elevation', y_axis_label='PM2.5 Level')
p2.circle(x='Elevation', y='PM25_MAR_8', 
         size=20, alpha=0.6, color='purple', source=bpm25)

show(p2)

### Can we build a model that uses *both* ``Elevation`` and ``Income`` to predict pollution levels?

In [24]:
mod = smf.ols(formula='PM25_MAR_8 ~ Elevation + Income + Elevation:Income', data=pm25)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             PM25_MAR_8   R-squared:                       0.580
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     27.61
Date:                Fri, 08 Jun 2018   Prob (F-statistic):           2.42e-11
Time:                        15:51:26   Log-Likelihood:                -69.505
No. Observations:                  64   AIC:                             147.0
Df Residuals:                      60   BIC:                             155.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           22.4080      6.418      3.492      0.001       9.571      35.245
Elevation           -0.0135      0.005     -2.811      0.007      -0.023      -0.004
Income              -0.0002      0.000     -1.624      0.110      -0.000    3.82e-05
Elevation:Income  1.034e-07   7.42e-08      1.393      0.169    -4.5e-08    2.52e-07
==============================================================================
Omnibus:                        3.191   Durbin-Watson:                   1.242
Prob(Omnibus):                  0.203   Jarque-Bera (JB):                2.305
Skew:                           0.386   Prob(JB):                        0.316
Kurtosis:                       3.517   Cond. No.                     4.74e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.74e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### What happens when we include both predictors?

# Now let's look at ozone

[Ozone](https://www.epa.gov/ozone-pollution) is another air quality concern. It is created by chemical reactions between oxides of nitrogen (NOx) and volatile organic compounds (VOC) in the presence of sunlight. Emissions from industrial facilities and electric utilities, vehicle exhaust, gasoline vapors, and chemical solvents are some of the major sources of NOx and VOC. Ground level ozone can cause breathing problems in sensitive populations and may also harm delicate ecosystems.

### We have ozone measurements at two times: 10:00 AM and 3:00 PM (15:00). Let's start by plotting both together to see how the timepoints differ.

In [54]:
boz = ColumnDataSource(ozone)

TOOLS = "box_select,lasso_select,help,reset"

left = figure(tools=TOOLS, plot_width=400, plot_height=400, x_axis_label='Income',
           y_axis_label='Ozone Level')
left.circle(x='Income', y='Ozone_AUG_10', size=20, alpha=0.6, color='navy', source=boz)
left.circle(x='Income', y='Ozone_AUG_15', size=20, alpha=0.6, color='green', source=boz)

right = figure(tools=TOOLS, plot_width=400, plot_height=400, x_axis_label='Elevation')
right.circle(x='Elevation', y='Ozone_AUG_10', size=20, alpha=0.6, color='navy', source=boz)
right.circle(x='Elevation', y='Ozone_AUG_15', size=20, alpha=0.6, color='green', source=boz)

p = gridplot([[left, right]])

show(p)

What might account for the difference in ozone levels between the two timepoints? (Click one of the options below.)

In [12]:
button1 = widgets.Button(description='a. Ozone levels decrease with elevation',layout=widgets.Layout(width='40%', height='40px'))
button2 = widgets.Button(description='b. Wealthy people spend more time outdoors',layout=widgets.Layout(width='40%', height='40px'))
button3 = widgets.Button(description='c. Ozone levels increase throughout the day',layout=widgets.Layout(width='40%', height='40px'))
button4 = widgets.Button(description='d. Too much noise; relationship is not obvious',layout=widgets.Layout(width='40%', height='40px'))
display(button1,button2,button3, button4)

button1.on_click(quiz.incorrect)
button2.on_click(quiz.incorrect)
button3.on_click(quiz.correct)
button4.on_click(quiz.incorrect)

Button(description='a. Ozone levels decrease with elevation', layout=Layout(height='40px', width='40%'), style=ButtonStyle())

Button(description='b. Wealthy people spend more time outdoors', layout=Layout(height='40px', width='40%'), style=ButtonStyle())

Button(description='c. Ozone levels increase with elevation, as does income', layout=Layout(height='40px', width='40%'), style=ButtonStyle())

Button(description='d. Too much noise; relationship is not obvious', layout=Layout(height='40px', width='40%'), style=ButtonStyle())

Not quite. Please try again.
Not quite. Please try again.


### Let's use the same statistical functions we used above to explore the relationship between ozone, income, and elevation.

In [26]:
mod = smf.ols(formula='Ozone_AUG_10 ~ Income', data=ozone)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Ozone_AUG_10   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     46.81
Date:                Fri, 08 Jun 2018   Prob (F-statistic):           4.04e-09
Time:                        15:51:38   Log-Likelihood:                -184.23
No. Observations:                  64   AIC:                             372.5
Df Residuals:                      62   BIC:                             376.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     20.0326      1.352     14.819      0.000      17.330      22.735
Income         0.0002   2.73e-05      6.841      0.000       0.000       0.000
==============================================================================
Omnibus:                        2.472   Durbin-Watson:                   1.552
Prob(Omnibus):                  0.291   Jarque-Bera (JB):                1.655
Skew:                           0.313   Prob(JB):                        0.437
Kurtosis:                       3.478   Cond. No.                     1.22e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#scatter with regression line

In [27]:
mod = smf.ols(formula='Ozone_AUG_10 ~ Elevation', data=ozone)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Ozone_AUG_10   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     25.86
Date:                Fri, 08 Jun 2018   Prob (F-statistic):           3.64e-06
Time:                        15:51:42   Log-Likelihood:                -191.07
No. Observations:                  64   AIC:                             386.1
Df Residuals:                      62   BIC:                             390.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -38.2037     13.128     -2.910      0.005     -64.447     -11.960
Elevation      0.0497      0.010      5.086      0.000       0.030       0.069
==============================================================================
Omnibus:                        4.418   Durbin-Watson:                   0.834
Prob(Omnibus):                  0.110   Jarque-Bera (JB):                2.072
Skew:                           0.071   Prob(JB):                        0.355
Kurtosis:                       2.130   Cond. No.                     2.90e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.9e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#scatter with regression line

### Based on these regression summaries, what would you say about the relationship between ozone and income, and ozone and elevation? Are they strong or weak relationships? 
   Try running the regression using the other timepoint (change *Ozone_AUG_10* to *Ozone_AUG_15*). How do the summaries change?

### Coming back to our original question, based on these analyses, do you think the air pollution is equally distributed across different socio-economic groups?